In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys, os
import pandas as pd

import logging
FORMAT = '%(asctime)s %(name)s %(funcName)s %(levelname)s %(message)s'
log_level = logging.WARNING
logging.basicConfig(format=FORMAT, datefmt='%H:%M:%S',
                    level=log_level)
plt.set_loglevel(level='warning')
logging.getLogger("umap").disabled = True

os.chdir('/scicore/home/nimwegen/degroo0000/Bonsai-data-representation')
sys.path.append('/scicore/home/nimwegen/degroo0000/Bonsai-data-representation')

from bonsai.bonsai_dataprocessing import loadReconstructedTreeAndData, Metadata
pd.set_option("display.max_rows", None)

In [ ]:
import json

In [ ]:
# tree_folder = '/scicore/home/nimwegen/GROUP/Projects/bonsai_runs/hao_satija_2021-CITEseq-immune_cells/run-10xv2/altered_output/bonsai-hao_annot_SUB-protein-cellstates_premerged_tree-rerun-updated_bonsai_code_nov24/results/final_bonsai_zscore1.0_tmpStartpremerged_tree'
tree_folder = '/scicore/home/nimwegen/GROUP/Projects/bonsai_runs/bonsai_scout_public/Cord_blood_cells_CITE-seq/Bonsai/final_bonsai_zscore1.0_tmpStartpremerged_tree'
tree_folder_sim = '/scicore/home/nimwegen/GROUP/Projects/bonsai_runs/bonsai_scout_public/Simulated_dataset_unbalanced_tree_with_random_branch_lengths_and_realistic_covariance/Bonsai/final_bonsai_zscore1.0_tmpStartpremerged_cs_tree'
ltqs_gv = np.load(os.path.join(tree_folder, 'posterior_ltqs_vertByGene.npy')).T
ltqs_vars_gv = np.load(os.path.join(tree_folder, 'posterior_ltqsVars_vertByGene.npy')).T
ltqs_gv_sim = np.load(os.path.join(tree_folder_sim, 'posterior_ltqs_vertByGene.npy')).T
ltqs_vars_gv_sim = np.load(os.path.join(tree_folder_sim, 'posterior_ltqsVars_vertByGene.npy')).T

In [ ]:
settings_path = os.path.join(tree_folder, 'metadata.json')
with open(settings_path, 'r') as json_file:
    self_dict = json.load(json_file)
bonvis_metadata = Metadata(json_filepath=settings_path)
gene_ids = bonvis_metadata.geneIds

In [ ]:
args = {'dataset': 'hao_satija_2021-CITEseq-immune_cells', 'verbose': True, 'filenames_data': 'delta_vmax.txt,d_delta_vmax.txt',
        'data_folder': 'dummy_path',
        'input_is_sanity_output': True,
        'results_folder': tree_folder,
        'rescale_by_var': True}
scData, vert_ind_to_node_id = loadReconstructedTreeAndData(args, tree_folder, reprocess_data=False, all_genes=False,
                                                           all_ranks=True,
                                                           get_cell_info=True, corrected_data=False,
                                                           rel_to_results=False,
                                                           no_data_needed=False,
                                                           single_process=False, keep_original_data=True,
                                                           calc_loglik=False, get_data=True,
                                                           get_posterior_ltqs=True)

In [ ]:
edgeInfo = pd.read_csv(os.path.join(tree_folder, 'edgeInfo.txt'), header=None, sep='\t')
edgeInfo.columns = ['source', 'target', 'length']
edgeInfo_sim = pd.read_csv(os.path.join(tree_folder_sim, 'edgeInfo.txt'), header=None, sep='\t')
edgeInfo_sim.columns = ['source', 'target', 'length']

In [ ]:
vertInfo = pd.read_csv(os.path.join(tree_folder, 'vertInfo.txt'), header=0, sep='\t')
vert_ids = list(vertInfo['vertName'])

In [ ]:
gene_diffs = np.zeros((ltqs_gv.shape[0], edgeInfo.shape[0]))
gene_diffs_sim = np.zeros((ltqs_gv_sim.shape[0], edgeInfo_sim.shape[0]))

In [ ]:
source_list = list(edgeInfo['source'])
dest_list = list(edgeInfo['target'])
for edge_ind, source in enumerate(source_list):
    dest = dest_list[edge_ind]
    gene_diffs[:, edge_ind] = ltqs_gv[:, dest] - ltqs_gv[:, source]

In [ ]:
source_list_sim = list(edgeInfo_sim['source'])
dest_list_sim = list(edgeInfo_sim['target'])
for edge_ind, source in enumerate(source_list_sim):
    dest = dest_list_sim[edge_ind]
    gene_diffs_sim[:, edge_ind] = ltqs_gv_sim[:, dest] - ltqs_gv_sim[:, source]

In [ ]:
corr_coeffs = np.corrcoef(gene_diffs)
corr_coeffs_sim = np.corrcoef(gene_diffs_sim)

In [ ]:
triu_indices = np.triu_indices(gene_diffs.shape[0], k=1)
triu_indices

In [ ]:
triu_indices_sim = np.triu_indices(gene_diffs_sim.shape[0], k=1)
triu_indices_sim

In [ ]:
corr_coeffs_flat = corr_coeffs[triu_indices[0], triu_indices[1]]
corr_coeffs_flat_sim = corr_coeffs_sim[triu_indices_sim[0], triu_indices_sim[1]]

In [ ]:
fig_hist, ax_hist = plt.subplots(ncols=2, figsize=(10,4))
ax_hist[0].hist(corr_coeffs_flat, bins=100, label='Stoeckius et al. dataset', density=True, log=False);
ax_hist[0].hist(corr_coeffs_flat_sim, bins=100, label='Simulated dataset', density=True, histtype='step', log=False);
ax_hist[0].set_xlabel('gene-gene correlation (PearsonR)')
ax_hist[0].legend();
ax_hist[0].set_ylabel('Probability density of gene-gene pairs');

ax_hist[1].hist(corr_coeffs_flat, bins=100, label='Stoeckius et al. dataset', log=True);
ax_hist[1].hist(corr_coeffs_flat_sim, bins=100, label='Simulated dataset', histtype='step', log=True);
ax_hist[1].set_xlabel('gene-gene correlation (PearsonR)');
ax_hist[1].set_ylabel('Number of gene-gene pairs');
ax_hist[1].legend();

plt.suptitle('Gene-gene correlations along the branches of the inferred Bonsai tree')

# plt.savefig("/scicore/home/nimwegen/degroo0000/bonsai-development/useful_scripts_not_bonsai/specific_analysis_scripts/jupyter_notebooks/figures/gene_correlations.png", dpi=300)
# plt.savefig("/scicore/home/nimwegen/degroo0000/bonsai-development/useful_scripts_not_bonsai/specific_analysis_scripts/jupyter_notebooks/figures/gene_correlations.svg")

# Look into some of the high correlation pairs

In [ ]:
corr_coeffs_flat_sorted_up = np.argsort(corr_coeffs_flat)
corr_coeffs_flat_sorted_down = np.argsort(-corr_coeffs_flat)

In [ ]:
R_val = []
rank = []
gene_1 = []
gene_1_ind = []
gene_2 = []
gene_2_ind = []
n_pairs = 0
rank_pair = 0
while n_pairs < 100:
    ind_pair = corr_coeffs_flat_sorted_down[rank_pair]
    gene_1_ind_pair = triu_indices[0][ind_pair]
    gene_2_ind_pair = triu_indices[1][ind_pair]
    gene_1_pair = gene_ids[gene_1_ind_pair]
    gene_2_pair = gene_ids[gene_2_ind_pair]
    if gene_1_pair.split('_trscrpt')[0] == gene_2_pair.split('_trscrpt')[0]:
        rank_pair += 1
        continue
    gene_1_ind.append(gene_1_ind_pair)
    gene_2_ind.append(gene_2_ind_pair)
    gene_1.append(gene_1_pair)
    gene_2.append(gene_2_pair)
    rank.append(rank_pair)
    R_val.append(corr_coeffs_flat[ind_pair])
    rank_pair += 1
    n_pairs += 1
pos_corr_dict = {'R_val': R_val, 'rank':rank, 'gene_1': gene_1, 'gene_2': gene_2, 'gene_1_ind': gene_1_ind, 'gene_2_ind': gene_2_ind}
pos_corr_df = pd.DataFrame(pos_corr_dict)
pos_corr_df

In [ ]:
R_val = []
rank = []
gene_1 = []
gene_1_ind = []
gene_2 = []
gene_2_ind = []
n_pairs = 0
rank_pair = 0
while n_pairs < 100:
    ind_pair = corr_coeffs_flat_sorted_up[rank_pair]
    gene_1_ind_pair = triu_indices[0][ind_pair]
    gene_2_ind_pair = triu_indices[1][ind_pair]
    gene_1_pair = gene_ids[gene_1_ind_pair]
    gene_2_pair = gene_ids[gene_2_ind_pair]
    if gene_1_pair.split('_trscrpt')[0] == gene_2_pair.split('_trscrpt')[0]:
        rank_pair += 1
        continue
    gene_1_ind.append(gene_1_ind_pair)
    gene_2_ind.append(gene_2_ind_pair)
    gene_1.append(gene_1_pair)
    gene_2.append(gene_2_pair)
    rank.append(rank_pair)
    R_val.append(corr_coeffs_flat[ind_pair])
    rank_pair += 1
    n_pairs += 1
neg_corr_dict = {'R_val': R_val, 'rank':rank, 'gene_1': gene_1, 'gene_2': gene_2, 'gene_1_ind': gene_1_ind, 'gene_2_ind': gene_2_ind}
neg_corr_df = pd.DataFrame(neg_corr_dict)
neg_corr_df

In [ ]:
# Check some correlations:
gene_pairs = [('BEST1_trscrpt4', 'FTH1_trscrpt2'), ('IGLL5', 'IGLC1'), ('AHSP_trscrpt2','GYPA_GYPB'), ('SDPR', 'HIST1H2AC')]
fig, axs = plt.subplots(nrows = len(gene_pairs), figsize=(4, 3* len(gene_pairs)))
for ind_gene, gene_pair1 in enumerate(gene_pairs):
    ax = axs[ind_gene]
    gene1_ind = gene_ids.index(gene_pair1[0])
    gene2_ind = gene_ids.index(gene_pair1[1])
    ax.scatter(ltqs_gv[gene1_ind, :], ltqs_gv[gene2_ind, :], s=2);
    ax.set_xlabel(gene_pair1[0]);
    ax.set_ylabel(gene_pair1[1]);
plt.tight_layout()